In [2]:
import os
import os.path
import requests 
from bs4 import BeautifulSoup 
import pygrib
import tarfile
#from pvlib.forecast import GFS
import pandas as pd
import numpy as np
import datetime
from pvlib.location import Location
import glob
import matplotlib.pyplot as plt
from pylab import rcParams
import xarray as xr
import itertools
%matplotlib inline

In [3]:
filename = 'gfs_3_20200201_0000_045.grb2'

with pygrib.open(filename) as grbs:
        grb = grbs.select(name = 'Downward short-wave radiation flux')[0]
        data, lats, lons = grb.data()
        
name2step = lambda name :pd.Timedelta(hours=int(name[-8:-5]))
base = datetime.datetime(year=grb.year, month=grb.month, day=grb.day, hour=grb.hour)
valid = base + name2step(filename)
validtime = np.repeat(pd.to_datetime(valid), (data.shape[0] * data.shape[1]))

In [5]:
data_flat = np.reshape(data, (np.product(data.shape),))

lats_flat = np.reshape(lats, (np.product(lats.shape),))
lons_flat = np.reshape(lons, (np.product(lons.shape),))

In [9]:
ds = xr.Dataset({'ghi': (['time'],data_flat)},
                coords={'lon': (['time'], lons_flat),
                        'lat': (['time'], lats_flat),
                       'time': (validtime)})


ds

<xarray.Dataset>
Dimensions:  (time: 65160)
Coordinates:
    lon      (time) float64 0.0 1.0 2.0 3.0 4.0 ... 356.0 357.0 358.0 359.0
    lat      (time) float64 90.0 90.0 90.0 90.0 90.0 ... -90.0 -90.0 -90.0 -90.0
  * time     (time) datetime64[ns] 2020-02-03T03:00:00 ... 2020-02-03T03:00:00
Data variables:
    ghi      (time) float32 0.0 0.0 0.0 0.0 0.0 ... 240.0 240.0 240.0 240.0